<a href="https://colab.research.google.com/github/thisIsVansh/AITP-Group-Assignment-1/blob/main/AITP_Group_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
# FINAL MARKET PULSE ENGINE (High-Contrast & Fixed Toggle)
# ==============================================================================

# 1. INSTALL DEPENDENCIES
!pip install -q gradio plotly vaderSentiment supabase pandas

import gradio as gr
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from supabase import create_client, Client
import pandas as pd

# 2. CONNECT TO SUPABASE
SUPABASE_URL = "https://ynuyfwalhuziobarzdbe.supabase.co"
SUPABASE_KEY = "sb_publishable_vMaWmbgDMbuMQR1Dd5WszA_eihZnoZA"

print("📡 Connecting to Supabase...")

try:
    supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

    # Fetch Data
    comments_data = supabase.table("comments").select("*").execute().data
    videos_data = supabase.table("videos").select("*").execute().data

    # Merge
    df_comments = pd.DataFrame(comments_data)
    df_videos = pd.DataFrame(videos_data)
    df = df_comments.merge(df_videos, on="video_id", how="left")
    print(f"✅ Successfully loaded {len(df)} rows.")

except Exception as e:
    print(f"⚠️ Connection Error: {e}")
    # Fallback Data
    df = pd.DataFrame({
        'content': ['Great phone!', 'Battery dies fast', 'Okay camera', 'Love it', 'Too expensive', 'Amazing screen', 'Laggy UI'],
        'published_at': ['2023-10-01', '2023-10-01', '2023-10-02', '2023-10-02', '2023-10-03', '2023-10-04', '2023-10-05']
    })

# 3. INTELLIGENCE LAYER
print("🧠 Running Analytics...")
analyzer = SentimentIntensityAnalyzer()
text_col = 'content' if 'content' in df.columns else ('comment_text' if 'comment_text' in df.columns else df.columns[0])

df['sentiment_score'] = df[text_col].apply(lambda x: analyzer.polarity_scores(str(x))['compound'])

# Date Parsing
date_col = 'published_at_x' if 'published_at_x' in df.columns else 'published_at'
df['date'] = pd.to_datetime(df[date_col], errors='coerce').dt.date
if df['date'].isnull().any():
    df['date'] = df['date'].fillna(pd.to_datetime('today').date())

# 4. AGGREGATION
daily_stats = df.groupby('date').agg(
    daily_sentiment=('sentiment_score', 'mean'),
    volume=('sentiment_score', 'count')
).reset_index().sort_values('date')
daily_stats['rolling_avg'] = daily_stats['daily_sentiment'].rolling(window=3, min_periods=1).mean()

# 5. UI FUNCTIONS
# ---------------------------------------------------------

def plot_advanced_trend(theme_mode):
    """Creates a Dual-Axis chart with Forced Theme Colors"""
    if daily_stats.empty: return None

    # FORCE COLORS based on toggle (Ignoring System Defaults)
    if theme_mode == "Dark":
        template = "plotly_dark"
        bar_color = 'rgba(255, 255, 255, 0.2)'
        line_color = '#818cf8' # Light Indigo
        trend_color = '#34d399' # Emerald
    else:
        template = "plotly_white"
        bar_color = 'rgba(0, 0, 0, 0.15)' # Darker grey for visibility
        line_color = '#4f46e5' # Deep Indigo
        trend_color = '#059669' # Deep Green

    # Create Figure
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # 1. Volume Bars
    fig.add_trace(
        go.Bar(x=daily_stats['date'], y=daily_stats['volume'], name="Vol", marker_color=bar_color),
        secondary_y=True
    )

    # 2. Raw Sentiment Dots
    fig.add_trace(
        go.Scatter(x=daily_stats['date'], y=daily_stats['daily_sentiment'], mode='markers', name="Daily", marker=dict(size=6, color=line_color)),
        secondary_y=False
    )

    # 3. Trend Line
    fig.add_trace(
        go.Scatter(x=daily_stats['date'], y=daily_stats['rolling_avg'], mode='lines', name="Trend", line=dict(width=4, color=trend_color)),
        secondary_y=False
    )

    fig.update_layout(
        title="Market Pulse Trend",
        template=template,
        margin=dict(l=20, r=20, t=50, b=20),
        legend=dict(orientation="h", y=1.1)
    )

    fig.update_yaxes(title_text="Sentiment", range=[-1.1, 1.1], secondary_y=False)
    fig.update_yaxes(showgrid=False, secondary_y=True) # Hide secondary grid to avoid clutter

    return fig

def render_reviews(theme_mode):
    """Generates High-Contrast HTML Cards"""

    top_pos = df.sort_values(by='sentiment_score', ascending=False).head(2)
    top_neg = df.sort_values(by='sentiment_score', ascending=True).head(2)

    # HARDCODED HIGH CONTRAST PALETTES
    if theme_mode == "Dark":
        # Dark Background, White Text
        bg_pos = "#064e3b" # Deep Green
        bg_neg = "#7f1d1d" # Deep Red
        text_main = "#ffffff" # Pure White
        text_sub = "#d1d5db" # Light Grey
        border_pos = "#34d399"
        border_neg = "#f87171"
    else:
        # White Background, Black Text (No Pastels)
        bg_pos = "#ffffff" # Pure White
        bg_neg = "#ffffff" # Pure White
        text_main = "#000000" # Pure Black
        text_sub = "#4b5563" # Dark Grey
        border_pos = "#166534" # Dark Green Border
        border_neg = "#991b1b" # Dark Red Border

    def create_card(row, bg, border, title_color):
        return f"""
        <div style="background-color:{bg}; color:{text_main}; padding:15px; margin-bottom:12px; border-radius:8px; border: 1px solid #e5e7eb; border-left: 6px solid {border}; box-shadow: 0 4px 6px -1px rgba(0, 0, 0, 0.1);">
            <div style="display:flex; justify-content:space-between; align-items:center; margin-bottom:8px;">
                <span style="font-weight:900; font-size:1.1em; color:{border};">Score: {row['sentiment_score']:.2f}</span>
                <span style="font-size:0.85em; color:{text_sub};">{row['date']}</span>
            </div>
            <div style="font-style:italic; line-height:1.5; font-weight:500;">"{row[text_col]}"</div>
        </div>
        """

    html = ""

    # Positive Section
    html += f"<h4 style='color:{border_pos if theme_mode=='Light' else '#4ade80'}; margin-top:10px; font-weight:bold;'>💚 Top Positive Reviews</h4>"
    for _, row in top_pos.iterrows():
        html += create_card(row, bg_pos, border_pos, "green")

    # Negative Section
    html += f"<h4 style='color:{border_neg if theme_mode=='Light' else '#f87171'}; margin-top:25px; font-weight:bold;'>💔 Top Negative Reviews</h4>"
    for _, row in top_neg.iterrows():
        html += create_card(row, bg_neg, border_neg, "red")

    return html

# 6. APP LAYOUT
# ---------------------------------------------------------
# We use 'Monochrome' theme as it is neutral and clashes least with our custom colors
with gr.Blocks(theme=gr.themes.Monochrome()) as app:

    with gr.Row():
        with gr.Column(scale=3):
            gr.Markdown("# 🚀 Market Pulse Engine")
        with gr.Column(scale=1):
            # The Toggle
            theme_selector = gr.Radio(["Light", "Dark"], label="Display Mode", value="Light")

    with gr.Tabs():
        with gr.TabItem("📈 Pulse Trends"):
            pulse_chart = gr.Plot(label="Market Analytics")
            gr.Markdown("*Tip: Bars = Volume, Line = Sentiment Trend*")
            refresh_btn = gr.Button("🔄 Refresh Data")

        with gr.TabItem("🗣️ Voice of Customer"):
            review_feed = gr.HTML()

    # Event Listeners (The "Brain" of the toggle)
    theme_selector.change(fn=plot_advanced_trend, inputs=theme_selector, outputs=pulse_chart)
    theme_selector.change(fn=render_reviews, inputs=theme_selector, outputs=review_feed)

    # Initial Load
    app.load(fn=plot_advanced_trend, inputs=theme_selector, outputs=pulse_chart)
    app.load(fn=render_reviews, inputs=theme_selector, outputs=review_feed)
    refresh_btn.click(fn=plot_advanced_trend, inputs=theme_selector, outputs=pulse_chart)

print("🚀 Launching High-Contrast Dashboard...")
app.launch(share=True, debug=False)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 kB 4.3 MB/s eta 0:00:00
📡 Connecting to Supabase...
✅ Successfully loaded 8 rows.
🧠 Running Analytics...


/tmp/ipython-input-2036948103.py:166: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Monochrome()) as app:


🚀 Launching High-Contrast Dashboard...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d2678659304786cfa0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
